In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pickle
import json

In [11]:
data_set = pickle.load(open('mushroom.p', 'rb'))
para = json.loads(open('para.json', 'r').read())

In [12]:
import tensorflow as tf

_, w, h, c = data_set['train_image'].shape
num_of_class = len(np.unique(data_set['train_label'])) # train_label에 등장하는 class의 개수로 전체 클래스의 개수를 추정
log = {
    'loss' : [],
    'training_accuracy' : [],
    'test_accuracy' : []
}

tf.reset_default_graph()

# hyper parameters
learning_rate = 0.001
n_epochs = 15
batch_size = 20

l_rate = para['learning_rate']
num_of_iteration = para['num_of_iteration']
print_frequency = para['print_frequency']

train_x = np.reshape(data_set['train_image'], (-1, w * h * c))
test_x = np.reshape(data_set['test_image'], (-1, w * h * c))
train_y = np.reshape(data_set['train_label'], (-1, 1))
test_y = np.reshape(data_set['test_label'], (-1, 1))

X = tf.placeholder(tf.float32, [None, w, h, c])
X_img = X
y = tf.placeholder(tf.int32, [None, 2])

# L1 ImgIn shape=(?, 32, 32, 3)
W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev=0.01)) #3*3*1 filter 32개 random 생성
# Conv -> (?, 32, 32, 32)
# Pool -> (?, 16, 16, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME') #strides: [number, height, width, channel] , padding='SAME': input size 유지
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], #kernel(filter) size = 2*2
                    strides=[1, 2, 2, 1], padding='SAME') #stride=2

# L2 ImgIn shape=(?, 16, 16, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) #3*3*32 filter 64개 random 생성
# Conv ->(?, 16, 16, 64)
# Pool ->(?, 8, 8, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

L2_flat = tf.reshape(L2, [-1, 8 * 8 * 64])  #FC input 형태로 reshape

# Final FC 7x7x64 inputs -> 10 outputs
W3 = tf.get_variable("W3", shape=[8 * 8 * 64, 2],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([2]))
logits = tf.matmul(L2_flat, W3) + b

# define cost/loss & optimizer
xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)) #index로 label 뽑음
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# initialize
init = tf.global_variables_initializer()

def batch_index(l, batch_size):
    a = l // batch_size
    b = np.random.permutation(l)
    return np.split(b, a)

# train my model
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for ind in batch_index(len(data_set['train_image']), batch_size):
            X_batch, y_batch = data_set['train_image'][ind], data_set['train_label'][ind]
            y_onehot = [[1,0] if i == 0 else [0,1] for i in y_batch]
            y_batch = np.array(y_onehot)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        test_y_onehot = [[1,0] if i == 0 else [0,1] for i in data_set['test_label']]
        test_y_batch = np.array(test_y_onehot)
        acc_test = accuracy.eval(feed_dict={X: data_set['test_image'], y: test_y_batch})
        print("Epoch:", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

Epoch: 0 Train accuracy: 0.7 Test accuracy: 0.67
Epoch: 1 Train accuracy: 0.9 Test accuracy: 0.71
Epoch: 2 Train accuracy: 0.75 Test accuracy: 0.715
Epoch: 3 Train accuracy: 0.7 Test accuracy: 0.775
Epoch: 4 Train accuracy: 0.95 Test accuracy: 0.83
Epoch: 5 Train accuracy: 0.9 Test accuracy: 0.82
Epoch: 6 Train accuracy: 1.0 Test accuracy: 0.865
Epoch: 7 Train accuracy: 1.0 Test accuracy: 0.82
Epoch: 8 Train accuracy: 1.0 Test accuracy: 0.88
Epoch: 9 Train accuracy: 1.0 Test accuracy: 0.87
Epoch: 10 Train accuracy: 1.0 Test accuracy: 0.87
Epoch: 11 Train accuracy: 1.0 Test accuracy: 0.88
Epoch: 12 Train accuracy: 1.0 Test accuracy: 0.875
Epoch: 13 Train accuracy: 1.0 Test accuracy: 0.875
Epoch: 14 Train accuracy: 0.9 Test accuracy: 0.83


In [9]:
1000 // 30

33

In [10]:
print(data_set['test_label'].shape)

(200,)


In [13]:
print(accuracy.eval(feed_dict={X: data_set['test_image'][0], y: test_y_batch[0]}))

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`